# Bidding

This is a tutorial for how to use the bidding engine

In [1]:
import sys
sys.path.append('../../src')

from nn.models import Models
from bots import BotBid
from util import hand_to_str
from sample import Sample
import conf
import numpy as np

np.set_printoptions(precision=2, suppress=True, linewidth=200)
np.random.seed(42)

models = Models.from_conf(conf.load('../Conf/UCBC2024.conf'),'..')   # loading neural networks
sampler = Sample.from_conf(conf.load('../Conf/UCBC2024.conf'), False)  # Load sampling strategies


Instructions for updating:
non-resource variables are not supported in the long term
INFO:tensorflow:Restoring parameters from ..\Models/NS1EW99-bidding_same-5556000
INFO:tensorflow:Restoring parameters from ..\Models/NS1EW99-binfo_same-5556000
INFO:tensorflow:Restoring parameters from ..\Models/lead_suit-999000
INFO:tensorflow:Restoring parameters from ..\Models/lead_nt-475000
INFO:tensorflow:Restoring parameters from ..\Models/Jack/lr3-1000000
INFO:tensorflow:Restoring parameters from ..\Models/single_dummy-32768000
INFO:tensorflow:Restoring parameters from ..\Models/lefty_nt-475000
INFO:tensorflow:Restoring parameters from ..\Models/dummy_nt-475000
INFO:tensorflow:Restoring parameters from ..\Models/righty_nt-475000
INFO:tensorflow:Restoring parameters from ..\Models/decl_nt-950000
INFO:tensorflow:Restoring parameters from ..\Models/lefty_suit-999000
INFO:tensorflow:Restoring parameters from ..\Models/dummy_suit-999000
INFO:tensorflow:Restoring parameters from ..\Models/righty_suit-

### Running through an example

In [2]:
# East deals, EW vulnerable.
vuln_ns, vuln_ew = True, True

# you sit West and hold:
hand = '85.9.KQJT873.QJT'

# the auction goes:
auction = []
bot_bid = BotBid([vuln_ns, vuln_ew], hand, models, sampler, 0, 0, False)

bid = bot_bid.bid(auction)
print(bid.to_dict()['candidates'])
auction = ["PAD_START"]
bot_bid = BotBid([vuln_ns, vuln_ew], hand, models, sampler, 1, 1, False)

bid = bot_bid.bid(auction)
print(bid.to_dict()['candidates'])
auction = ["PAD_START","PAD_START"]
bot_bid = BotBid([vuln_ns, vuln_ew], hand, models, sampler, 2, 2, False)

bid = bot_bid.bid(auction)
print(bid.to_dict()['candidates'])
auction = ["PAD_START","PAD_START","PAD_START"]
bot_bid = BotBid([vuln_ns, vuln_ew], hand, models, sampler, 3, 3, False)

bid = bot_bid.bid(auction)
print(bid.to_dict()['candidates'])

# what's your bid?

[{'call': '3D', 'insta_score': 0.897}, {'call': '2D', 'insta_score': 0.083}]
[{'call': '3D', 'insta_score': 0.945}]
[{'call': '3D', 'insta_score': 0.897}, {'call': '2D', 'insta_score': 0.083}]
[{'call': '3D', 'insta_score': 0.945}]


In [3]:
auction = ["PAD_START", "2D", "PASS", "PASS", "X", "PASS", "PASS"]
bid = bot_bid.bid(auction)

bid.to_dict()['candidates']

[{'call': 'PASS',
  'insta_score': 0.994,
  'expected_score': 404,
  'adjustment': 497},
 {'call': '3D',
  'insta_score': 0.003,
  'expected_score': -207,
  'adjustment': -620}]

in the output above:
- `insta_score` reflects what the neural network prefers to bid
- `expected_score` is based on what you expect to score on difference samples (for example, 70% of the time you are making and 30% of the time you are going one off, then the expected score is 0.7 * 620 - 0.3 * 100)

### Samples consistent with the auction so far

Above we said that we are computig expected scores on samples consistent with the auction so far.

How do we get these samples?

We get some samples in the response we get from the bot (i.e `bid.samples`)

In [4]:
bid.samples

['AQ9.AKx..AK9xxxx KJT.8xx.A9xxxx.8 xxxxx.QJTxxx..xx 8x.9.KQJT8xx.QJT 0.73732 \n 2D-P-P-X-P-P-P (9.6, 9.3)  \n 2D-P-P-X-P-P-3D-P-P-4C-P-P-P (11.0, 11.8) ',
 'AQJT.AKTxx..AK8x xx.QJx.A9xxxx.9x K9xxx.8xxx..xxxx 8x.9.KQJT8xx.QJT 0.72674 \n 2D-P-P-X-P-P-P (8.8, 8.8)  \n 2D-P-P-X-P-P-3D-P-P-4H-P-P-P (11.7, 12.0) ',
 'AQT9xx.AJ8..AK8x Kxx.xxx.A9xxxx.x Jx.KQTxxx..9xxxx 8x.9.KQJT8xx.QJT 0.71237 \n 2D-P-P-X-P-P-P (9.5, 9.3)  \n 2D-P-P-X-P-P-3D-P-P-3S-P-P-P (11.3, 12.1) ',
 'AKJx.KQJ8..Axxxx Txx.x.A9xxxx.K9x Q9xx.ATxxxxx..8x 8x.9.KQJT8xx.QJT 0.70874 \n 2D-P-P-X-P-P-P (9.1, 9.4)  \n 2D-P-P-X-P-P-3D-P-P-4C-P-P-P (9.2, 12.1) ',
 'AKQx.QTxxx..AKxx Txx.AJ.A9xxxx.xx J9xx.K8xxx..98xx 8x.9.KQJT8xx.QJT 0.69171 \n 2D-P-P-X-P-P-P (9.3, 9.2)  \n 2D-P-P-X-P-P-3D-P-P-3H-P-P-P (11.4, 11.5) ',
 'KQT.AKQxx..AK8xx AJx.xx.A9xxxx.xx 9xxxx.JT8xx..9xx 8x.9.KQJT8xx.QJT 0.68798 \n 2D-P-P-X-P-P-P (9.6, 9.8)  \n 2D-P-P-X-P-P-3D-P-P-4C-P-P-P (10.2, 11.2) ',
 'AQxxx.AKQJx..K9x x.T8x.A9xxxx.Axx KJT9x.xxxx..8xxx 8x.9.KQJT8xx

each row is one sample. the hands are in the order: N E S W

(West always has the same cards because it's your hand and it's known to you)

In [5]:
# we got some samples above. if we want more, we can always get more

sample_hands, sorted_score, p_hcp, p_shp, good_quality = bot_bid.sample_hands_for_auction(["PAD_START", "PAD_START", "PAD_START", "PASS", "1N", "PASS", "3S","PASS"],0)
print("Good quality:", good_quality)
for i in range(sample_hands.shape[0]):
    print(f'{hand_to_str(sample_hands[i, 0, :])} {hand_to_str(sample_hands[i, 1, :])} {hand_to_str(sample_hands[i, 2, :])} {hand_to_str(sample_hands[i, 3, :])}')

Good quality: True
8x.9.KQJT8xx.QJT xx.Qxxx.xx.9xxxx AKQJTxx.AJx.A.AK 9x.KT8xx.9xx.8xx
8x.9.KQJT8xx.QJT J9x.8xx.9xx.AKxx AKQxx.AKQJxx.x.9 Txx.Txx.Ax.8xxxx
8x.9.KQJT8xx.QJT 9xx.Txxx.xxx.98x AKQJT.AKQ8..Kxxx xxx.Jxxx.A9x.Axx
8x.9.KQJT8xx.QJT Jxx.Txxx.xx.A9xx AKQ9xx.AKQJ.A9.K Tx.8xxx.xx.8xxxx
8x.9.KQJT8xx.QJT Txx.K8xx.9x.9xxx AKQJ9.AJTxx.A.Ax xxx.Qxx.xxx.K8xx
8x.9.KQJT8xx.QJT Kxxx.Jxxx.xx.9xx AJT9x.AQT8.A.A8x Qx.Kxxx.9xx.Kxxx
8x.9.KQJT8xx.QJT KTx.T8x.A9x.xxxx AQJ9x.AKQJx..8xx xxx.xxxx.xxx.AK9
8x.9.KQJT8xx.QJT Axxx.J8xx.xx.9xx KQT9x.AKQx.A.K8x Jx.Txxx.9xx.Axxx
8x.9.KQJT8xx.QJT x.JTxx.9xxx.98xx AKQJTxx.AKQx..Ax 9xx.8xxx.Ax.Kxxx
8x.9.KQJT8xx.QJT QTxx.Kxxx.9x.xxx AKJ9x.AQJx.A.Kxx xx.T8xx.xxx.A98x
